In [38]:
import pandas as pd

In [39]:
data = pd.read_csv('data/wine_dataset.csv')
data.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,style
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,red
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,red
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,red
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,red
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,red


In [40]:
data.shape

(6497, 13)

In [41]:
data.dtypes

fixed_acidity           float64
volatile_acidity        float64
citric_acid             float64
residual_sugar          float64
chlorides               float64
free_sulfur_dioxide     float64
total_sulfur_dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
quality                   int64
style                    object
dtype: object

In [42]:
data.isnull().sum()

fixed_acidity           0
volatile_acidity        0
citric_acid             0
residual_sugar          0
chlorides               0
free_sulfur_dioxide     0
total_sulfur_dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
style                   0
dtype: int64

In [43]:
y = data['style']
x = data.drop('style', axis=1)

In [44]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
import numpy as np

# 1. Split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)

# 2. Scale the data (normalize the features)
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)  # Fit the scaler to the training data
x_test_scaled = scaler.transform(x_test)  # Apply the transformation to the test data

# 3. Define the parameter grid for logistic regression
c_values = np.array([0.01, 0.1, 0.5, 1, 2, 3, 5, 10, 20, 50, 100])

# 4. Create a grid for L1 and L2 penalties with appropriate solvers
param_grid = [
    {'C': c_values, 'penalty': ['l1'], 'solver': ['liblinear']},  # For L1 penalty
    {'C': c_values, 'penalty': ['l2'], 'solver': ['lbfgs']},      # For L2 penalty
    {'C': c_values, 'penalty': ['l2'], 'solver': ['saga']}        # For L2 penalty with the 'saga' solver
]

# 5. Create the logistic regression model
model = LogisticRegression(max_iter=10000, class_weight='balanced')

# 6. Perform GridSearchCV with StratifiedKFold (to ensure balanced class splits)
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(model, param_grid, cv=stratified_kfold, scoring='accuracy')
grid_search.fit(x_train_scaled, y_train)

# 7. Display the best hyperparameters found
print(f"Best Training Accuracy (with cross-validation): {grid_search.best_score_}")
print(f"Best C value: {grid_search.best_estimator_.C}")
print(f"Best penalty type: {grid_search.best_estimator_.penalty}")

# 8. Evaluate the performance on the test set
test_accuracy = grid_search.score(x_test_scaled, y_test)
print(f"Test set accuracy: {test_accuracy}")


Best Training Accuracy (with cross-validation): 0.994941186426335
Best C value: 2.0
Best penalty type: l2
Test set accuracy: 0.9902564102564102


### Correto

In [51]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import StratifiedKFold, cross_val_score

model = GaussianNB()
skfold = StratifiedKFold(n_splits=3)
result = cross_val_score(model, x_train_scaled, y_train, cv=skfold)
print(f"Cross-validation mean accuracy: {result.mean()}")

model.fit(x_test_scaled, y_test)

accuracy_model = model.score(x_test_scaled, y_test)
print(f"Test set accuracy: {accuracy_model}")


Cross-validation mean accuracy: 0.9771275808319618
Test set accuracy: 0.9702564102564103
